In [1]:
import numpy as np
import pandas as pd
from random import sample, seed
from itertools import combinations

def m2(x):
    A = np.append(np.ones((len(x),1)),x,axis=1)
    comb = combinations([0, 1, 2, 3, 4, 5, 6, 7], 2)
    for i in comb:
        A = np.append(A, (x[:,i[0]]*x[:,i[1]]).reshape(-1,1),axis = 1)
    return A

def m1(x):
    A = np.append(np.ones((len(x),1)),x,axis=1)
    return A

def m1_v1(x, d1):
    A = np.ones((len(x),1))
    A = np.append(A, x[:,d1].reshape(-1,1), axis = 1)
    return A

def m1_sigmoid(x):
    A = 1/(1+np.exp(-m1(x)))
    return A

def m1_gaussian(x):
    top = (x - np.mean(x, axis = 0))**2
    down = 2*(np.std(x, axis=0)**2)
    return np.exp(-(top/down))
        
    
def lr(x,y):
    A = x
    AT = A.T
    ATA = AT@A
    ATA_inv_ATb = np.linalg.pinv(ATA)@AT@y
    return ATA_inv_ATb
    
def lr_m1(x, y, d1):
    x1 = x[:,d1].reshape(-1,1)
    A = m1_v1(x, d1)
    AT = A.T
    ATA = AT@A
    ATA_inv_ATb = np.linalg.pinv(ATA)@AT@y
    return ATA_inv_ATb

def lr_map(x, y , l):
    A = x
    AT = A.T
    ATA = AT@A
    L = l*np.identity(x.shape[1])
    ATA_L = ATA + L
    ATA_inv_ATb = np.linalg.pinv(ATA_L)@AT@y
    return ATA_inv_ATb

def cross_val(x_train, y_train, n_fold, m):
    err = 0
    for i in range(n_fold):
        val_i = [i for i in range(int(len(x_train)/n_fold*i),int(len(x_train)/n_fold*(i+1)))]
        trn_i = list(set(i for i in range(len(x_train)))-set(val_i))
        if m == 1:
            x_mx = m1(x_train)
        elif m == 2:
            x_mx = m2(x_train)
        else:
            print("Unvalid parameter")
        w = lr(x_mx[trn_i], y_train[trn_i])
        err += rmse(w, x_mx[val_i], y_train[val_i])
    err = err/n_fold
    return err

def rmse(w, x, y):
    return np.sqrt(np.mean((x@w-y)**2))

In [2]:
df_x = pd.read_csv('data_X.csv')
df_y = pd.read_csv('data_T.csv')

seed(100)
np_x = np.array(df_x)
np_y = np.array(df_y)
trn_i = sample(range(len(np_x)),int(len(np_x)*0.8))
tst_i = list(set(i for i in range(len(np_x)))-set(trn_i))

## 1.a RMSE of m=1 and m=2

In [3]:
x_trn = m1(np_x[trn_i])
y_trn = np_y[trn_i]
x_tst = m1(np_x[tst_i])
y_tst = np_y[tst_i]

w = lr(x_trn, y_trn)
print("RMSE of training data with M=1:",rmse(w, x_trn, y_trn))
print("RMSE of testing data with M=1:",rmse(w, x_tst, y_tst))

RMSE of training data with M=1: 69947.54503942489
RMSE of testing data with M=1: 67987.91485157181


In [4]:
x_trn = m2(np_x[trn_i])
y_trn = np_y[trn_i]
x_tst = m2(np_x[tst_i])
y_tst = np_y[tst_i]

w = lr(x_trn, y_trn)
print("RMSE of training data with M=2:",rmse(w, x_trn, y_trn))
print("RMSE of testing data with M=2:",rmse(w, x_tst, y_tst))

RMSE of training data with M=2: 68571.39755855715
RMSE of testing data with M=2: 66751.88323947506


## 1.b Feature selection

In [5]:
for i in range(8):
    re = lr_m1(np_x[trn_i], np_y[trn_i], i)
    x = m1_v1(np_x, i)
    print("m=1 linear regression with feature",df_x.columns[i])
    print("RMSE of training data:",rmse(re,x[trn_i],np_y[trn_i]))
    print("RMSE of testing data:",rmse(re,x[tst_i],np_y[tst_i]))
    print("-------------------------")

m=1 linear regression with feature longitude
RMSE of training data: 115896.93215766865
RMSE of testing data: 112989.33074355309
-------------------------
m=1 linear regression with feature latitude
RMSE of training data: 114640.84539065433
RMSE of testing data: 112546.30229815214
-------------------------
m=1 linear regression with feature housing_median_age
RMSE of training data: 115289.42409690676
RMSE of testing data: 112723.2620104834
-------------------------
m=1 linear regression with feature total_rooms
RMSE of training data: 114903.0752862587
RMSE of testing data: 112397.4906748178
-------------------------
m=1 linear regression with feature total_bedrooms
RMSE of training data: 115823.50939163893
RMSE of testing data: 113148.63563745568
-------------------------
m=1 linear regression with feature population
RMSE of training data: 115939.21987036939
RMSE of testing data: 113211.91065070733
-------------------------
m=1 linear regression with feature households
RMSE of training 

## 2.b guassian Basis function 

In [6]:
x_trn = m1_gaussian(np_x[trn_i])
y_trn = np_y[trn_i]
x_tst = m1_gaussian(np_x[tst_i])
y_tst = np_y[tst_i]

w = lr(x_trn, y_trn)
print("RMSE of training data with gaussian basis function:",rmse(w, x_trn, y_trn))
print("RMSE of testing data with gaussian basis function:",rmse(w, x_tst, y_tst))

RMSE of training data with gaussian basis function: 116437.11495491366
RMSE of testing data with gaussian basis function: 113731.68364539843


## 2.c N-fold Cross validation

In [7]:
print("Average RMSE of 5-fold cross validation with m=1:",cross_val(np_x[trn_i], np_y[trn_i], 5, 1))
print("Average RMSE of 5-fold cross validation with m=2:",cross_val(np_x[trn_i], np_y[trn_i], 5, 2))

Average RMSE of 5-fold cross validation with m=1: 70151.4693429532
Average RMSE of 5-fold cross validation with m=2: 69603.9663386613


## 3.b MAP with gaussian

In [9]:
x_trn = m1_gaussian(np_x[trn_i])
y_trn = np_y[trn_i]
x_tst = m1_gaussian(np_x[tst_i])
y_tst = np_y[tst_i]

w = lr_map(x_trn, y_trn, 1)
print("RMSE of training data with MAP:",rmse(w, x_trn, y_trn))
print("RMSE of testing data with MAP:",rmse(w, x_tst, y_tst))

RMSE of training data with MAP: 116437.47128591538
RMSE of testing data with MAP: 113741.02810992872
